# Python Cosmic-ray Neutron Calibration Package

A series of python routines to help researchers and practicioners convert cosmic-ray neutron observations into volumetric water content. The package includes a routine for calibration of detectors using field observations. The package offers the following capabilities:

- Load data from different detectors
- Fill incomplete counts and flag spurious data
- Correct raw neutron counts 
- Filter corrected neutron counts
- Convert corrected and filtered neutron counts into volumetric water content
- Compute sensing depth
- Estiamte soil water storage using sensing depth and a user-defined fixed soil depth
- Calibrate a cosmic-ray neutron detector using depth weighed field observations

Function documentation follows PEP 257 docstring conventions.


**User-defined variables**

`Pref`: Reference barometric pressure in mbars

`L`: Atmospheric attenuation coefficient in mbar^-1 (see Dong et al. 2014 p.4)

`Aref`: Reference absolute air humidity in g/m^3

**Correction variables**

**Calibration variables**

In [1]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import warnings
import requests
import io
from bokeh.plotting import figure, show, output_notebook
output_notebook()


Loading BokehJS ...

In [2]:
def load_data(file_name,skip_rows,variables,round_time=True):
    """Function that reads a .csv file with raw neutron counts and associated atmospheric variables.
    
    Keyword arguments:
    
    file_name -- Name of file with tabular data. 
                 Must be a comma-separated value (.csv, .txt, .dat)
    skip_rows -- Positive integer representing the number of lines to skip (e.g., comments)
    variables -- Dictionary of lists containing the column position for each pre-defined variable.
                 Column values are 1-index to help users determine the position of each variable.
                 Only pre-defined variables will be retained. This is to ensure that the function
                 can be used with multiple devices and to remove the need to alter headers and 
                 column names in raw data files.
    """
    
    # Extract column names and rename if necessary (eg. more than one detector)
    col_names = []
    col_positions = []
    for var in variables.keys():
        for k,col in enumerate(variables[var]):
            col_positions.append(col-1) # Variable positions are 1-index
            if len(variables[var])>1:
                col_names.append(var + '_' + str(k+1))
            else:
                col_names.append(var)

    # Read data from .CSV file
    df = pd.read_csv(file_name, skiprows=skip_rows, usecols=col_positions)
    
    # Re-order column names and assign name to column
    df.columns = [x for _,x in sorted(zip(col_positions, col_names))]
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y/%m/%d %H:%M:%S')
    
    # Round timestamps to nearest hour
    if round_time:
        df['timestamp'] = df['timestamp'].dt.round('H')
    
    return df


In [3]:
def fill_counts(df,threshold=0.25):
    """Function to fill in hours with neutron counts in periods shorter than the intergration time.
    The function only fills rows with counts >1/4 of the integration time. Shorter periods are
    replaced with NaN. The 1/4 threshold is arbitrary, but it provides a reasonable threshold to
    avoid interpolations outside the typical range.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct
    threshold -- Minimum fraction of the neutron integration time to consider when filling rows
                 with incomplete time intervals. A value of 0 would enable interpolation with any
                 value of counting time [Default is 0.25]
    """
    
    # Find columns with count values and count time
    cols_count = [col for col in df.columns if "counts_raw" in col]
    cols_time = [col for col in df.columns if "counts_time" in col]
    time_threshold = count_time*threshold

    if len(cols_count) == len(cols_time):
        try:
            for k in range(len(cols_count)):
                idx_nan = df[cols_time[k]] < time_threshold
                df.loc[idx_nan,cols_count[k]] = np.nan
                df.loc[idx_nan,cols_time[k]] = np.nan

                idx_fill = (data[cols_time[k]] >= time_threshold) & (df[cols_time[k]] < count_time)
                df.loc[idx_fill,cols_count[k]] = (df.loc[idx_fill,cols_count[k]] * count_time / df.loc[idx_fill,cols_time[k]]).round()
                df.loc[idx_fill,cols_time[k]]  = count_time

            return df
        
        except ValueError as error:
            logger.error(error)
            raise
    else:
        raise ValueError("Number of column counts and number of column times don't match")
        

In [4]:
def compute_total_counts(df):
    """Compute the sum of uncorrected neutron counts for all detectors.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct.
    """
    
    cols_count = [col for col in df.columns if "counts_raw" in col]
    df['counts_total_raw'] = df[cols_count].sum(skipna=True, axis=1, min_count=1)
    return df


In [5]:
def fill_missing_atm(df,limit=24):
    """Fill missing values in atmospheric variables. Gap filling is performed using a 
    piecewise cubic Hermite interpolating polynomial (pchip method) that is restricted to intervals
    of missing data with a limited number of values and surrounded by valid observations.
    There is no interpolation at the end of the time series.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct.
    limit -- Positive integer denoting the largest gap in missing data that will be interpolated.
    """
    
    df['temperature'].interpolate(method='pchip', limit_area='inside', limit=limit, inplace=True)
    df['humidity'].interpolate(method='pchip', limit_area='inside', limit=limit, inplace=True)
    df['pressure'].interpolate(method='pchip', limit_area='inside', limit=limit, inplace=True)
    return df


In [6]:
def atm_correction(df,par):
    """Correct neutron counts for atmospheric factors and incoming neutron flux.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct.
    par -- User-defined arguments for the particular experiment or field
    
    References:
    Zreda, M., Shuttleworth, W. J., Zeng, X., Zweck, C., Desilets, D., Franz, T., et al. (2012). 
    COSMOS: the cosmic-ray soil moisture observing system. Hydrol. Earth Syst. Sci. 16, 4079–4099.
    doi: 10.5194/hess-16-4079-2012
    
    Andreasen, M., Jensen, K.H., Desilets, D., Franz, T.E., Zreda, M., Bogena, H.R. and Looms, M.C., 2017. 
    Status and perspectives on the cosmic‐ray neutron method for soil moisture estimation and other environmental science applications. 
    Vadose zone journal, 16(8), pp.1-11. doi.org/10.2136/vzj2017.04.0086
    """
       
    ### Barometric pressure factor
    fp = np.exp((df['pressure'] - par['Pref'])/par['L']); # Pressure factor (fp) [Eq. 1] Hawdon et al. 2014
    
    ### Atmospheric water vapor factor
    # Saturation vapor pressure 
    e_sat = 0.611*np.exp(17.502*df['temperature'] / (df['temperature']+240.97))*1000 # in Pascals Eq. 3.8 p.41 Environmental Biophysics (Campbell and Norman)
    
    # Vapor pressure Pascals
    Pw =  e_sat * df['humidity']/100; 
    
    # Absolute humidity (g/m^3)
    C = 2.16679 # g K/J;
    A = C * Pw / (df['temperature']+273.15) 
    fw = 1 + 0.0054*(A - par['Aref']);

    ### Incoming neutron flux factor
    if 'incoming_counts' not in df.columns:
        fi = 1
        warnings.warn("Ignoring incoming neutron flux correction factor (using value fi=1)")
    else:
        fi = df['incoming_counts']/df['incoming_counts'].iloc[0] # Use first value as reference
        fi[np.isnan(fi)] = 1 # Use a value of 1 for days without data 

    # Apply correction factors
    df['counts_total_corrected'] = np.round(df['counts_total_raw']*(fp*fw/fi))
    
    return df


In [7]:
def get_incoming_neutron_flux(df,par):
    """Function to retrieve neutron flux from the Neutron Monitor Database.
    Documentation available:https://www.nmdb.eu/nest/help.php#howto"""

    # Example: get_incoming_flux(station='IRKT',start_date='2020-04-10 11:00:00',end_date='2020-06-18 17:00:00')
    # Template url = 'http://nest.nmdb.eu/draw_graph.php?formchk=1&stations[]=KERG&output=ascii&tabchoice=revori&dtype=corr_for_efficiency&date_choice=bydate&start_year=2009&start_month=09&start_day=01&start_hour=00&start_min=00&end_year=2009&end_month=09&end_day=05&end_hour=23&end_min=59&yunits=0'

    start_date = data['timestamp'].iloc[0]
    end_date = data['timestamp'].iloc[-1]
    
    # Add 1 hour to ensure the last date is included in the request.
    end_date += pd.Timedelta('1H') 

    # Convert local time to UTC
    if par['utc_offset']>0:
        start_date -= pd.Timedelta( str(par['utc_offset']) + 'H')
        end_date -= pd.Timedelta( str(par['utc_offset']) + 'H')
    else:
        start_date += pd.Timedelta( str(par['utc_offset']) + 'H')
        end_date += pd.Timedelta( str(par['utc_offset']) + 'H')

    date_format = '%Y-%m-%d %H:%M:%S'
    root = 'http://www.nmdb.eu/nest/draw_graph.php?'
    url_par = [ 'formchk=1',
                'stations[]=' + par['nmdb_station'],
                'output=ascii',
                'tabchoice=revori',
                'dtype=corr_for_efficiency',
                'tresolution=' + str(60),
                'date_choice=bydate',
                'start_year=' + str(start_date.year),
                'start_month=' + str(start_date.month),
                'start_day=' + str(start_date.day),
                'start_hour=' + str(start_date.hour),
                'start_min=' + str(start_date.minute),
                'end_year=' + str(end_date.year),
                'end_month=' + str(end_date.month),
                'end_day=' + str(end_date.day),
                'end_hour=' + str(end_date.hour),
                'end_min=' + str(end_date.minute),
                'yunits=0' ]

    url = root + '&'.join(url_par)
    r = requests.get(url).content.decode('utf-8')

    # Subtract 1 hour to restore the last date is included in the request.
    end_date -= pd.Timedelta('1H') 

    start = r.find('\n' + start_date.strftime(format=date_format))
    end = r.find('\n' + end_date.strftime(format=date_format)) - 1
    s = r[start:end]
    s = r[start:end]
    s2 = ''.join([row.replace(';',',') for row in s])
    df_flux = pd.read_csv(io.StringIO(s2), names=['timestamp','counts'])
    df['incoming_counts'] = df_flux['counts']

    # Print acknowledgement to inform users about restrictions and to acknowledge the NMDB database
    acknowledgement = """Data retrieved via NMDB are the property of the individual data providers. These data are free for non commercial
use to within the restriction imposed by the providers. If you use such data for your research or applications, please acknowledge
the origin by a sentence like 'We acknowledge the NMDB database (www.nmdb.eu) founded under the European Union's FP7 programme 
(contract no. 213007), and the PIs of individual neutron monitors at: IGY Jungfraujoch 
(Physikalisches Institut, University of Bern, Switzerland)"""
    print(acknowledgement)

    return df


In [8]:
def smooth_counts(df,window=11,order=3):
    """Use a Savitzky-Golay filter to smooth the signal of corrected neutron counts.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct
    window --  Window of the filter, typically odd integer [Default is 11]
    order -- Order of the polynomial. [Default is 3]
    
    References:
    Franz, T.E., Wahbi, A., Zhang, J., Vreugdenhil, M., Heng, L., Dercon, G., Strauss, P., Brocca, L. and Wagner, W., 2020. 
    Practical data products from cosmic-ray neutron sensing for hydrological applications. Frontiers in Water, 2, p.9.
    doi.org/10.3389/frwa.2020.00009
    """
    
    # Smooth data
    mode = 'interp'
    df['counts_total_filtered'] = np.round(savgol_filter(df['counts_total_corrected'], window, order, mode=mode))

    return df


In [9]:
def counts_to_vwc(df,a0=0.0808,a1=0.372,a2=0.115):
    """Function to convert corrected and filtered neutron counts into volumetric water content
    following the approach described in Desilets et al., 2010.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct
    a0, a1, a2 -- Parameters given in Zreda et al., 2012.
    
    References: 
    Desilets, D., M. Zreda, and T.P.A. Ferré. 2010. Nature’s neutron probe: 
    Land surface hydrology at an elusive scale with cosmic rays. Water Resour. Res. 46:W11505.
    doi.org/10.1029/2009WR008726
    """
    
    # Convert neutron counts into vwc
    df['vwc'] = (a0 / (df['counts_total_filtered']/par['N0']-a1) - a2 - par['Wlat'] - par['Wsoc'])*par['bulk_density']
    return df


In [10]:
def fill_missing_vwc(df,limit=24):
    """Fill missing values in volumetric water content using a piecewise cubic Hermite 
    interpolating polynomial (pchip method)

    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct
    limit -- Maximum number of consecutive missing values to fill
    """
    
    # Interpolate rows with missing volumetric water content.
    df['vwc'].interpolate(method='pchip', limit_area='inside', limit=limit, inplace=True)
    return df


In [11]:
def sensing_depth(df,par,method='Schron_2017',dist=[1]):
    """Function that computes the estimated sensing depth of the cosmic-ray neutron probe.
    The function offers several methods to compute the depth at which 86 % of the neutrons
    probes the soil profile.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct.
    par -- User-defined arguments for the particular experiment or field
    method -- 'Schron_2017' or 'Franz_2012'
    dist -- List of radial distances at which to estimate the sensing depth. Only used for the 'Schron_2017' method.
    
    References:
    Franz, T.E., Zreda, M., Ferre, T.P.A., Rosolem, R., Zweck, C., Stillman, S., Zeng, X. and Shuttleworth, W.J., 2012.
    Measurement depth of the cosmic ray soil moisture probe affected by hydrogen from various sources.
    Water Resources Research, 48(8). doi.org/10.1029/2012WR011871
    
    Schrön, M., Köhli, M., Scheiffele, L., Iwema, J., Bogena, H. R., Lv, L., et al. (2017). 
    Improving calibration and validation of cosmic-ray neutron sensors in the light of spatial sensitivity. 
    Hydrol. Earth Syst. Sci. 21, 5009–5030. doi.org/10.5194/hess-21-5009-2017
    """
    
    # Determine sensing depth (D86)
    if method == 'Schron_2017':
        
        # See Appendix A of Schrön et al. (2017)
        Fp = 0.4922 / (0.86 - np.exp(-1*df['pressure']/par['Pref']));
        Fveg = 0
        
        for d in dist:
            # Compute r_star
            r_start = d/Fp 
            
            # Compute soil depth that accounts for 86% of the neutron flux
            col_name = 'sensing_depth_D86_' + str(d) + 'm'
            df[col_name] = 1/par['bulk_density'] * (8.321+0.14249*(0.96655 + np.exp(-0.01*r_start))*(20+(par['Wlat']+df['vwc'])) / (0.0429+(par['Wlat']+df['vwc'])))

    elif method == 'Franz_2012':
        df['sensing_depth_D86'] = 5.8/(par['bulk_density']*par['Wlat']+ df['vwc']+0.0829)
        
    return df
    

In [34]:
def compute_storage(df,T=1,Zsurface=150,Zrootzone=1000):
    """Exponential filter to estimate soil moisture in the rootzone from surface observtions.
    
    Keyword arguments:
    
    df -- This is the main DataFrame with tabular data to correct.
    Zsurface -- Depth of surface layer in mm. This should be an intermediate value according to the 
                sensing depth computed using the D86 method.
    T -- Characteristic time length in the same units as the measurement interval    
    
    References:
    Albergel, C., Rüdiger, C., Pellarin, T., Calvet, J.C., Fritz, N., Froissard, F., Suquia, D., Petitpa, A., Piguet, B. and Martin, E., 2008. 
    From near-surface to root-zone soil moisture using an exponential filter: an assessment of the method based on in-situ observations and model simulations. 
    Hydrology and Earth System Sciences, 12(6), pp.1323-1337.
    
    Franz, T.E., Wahbi, A., Zhang, J., Vreugdenhil, M., Heng, L., Dercon, G., Strauss, P., Brocca, L. and Wagner, W., 2020. 
    Practical data products from cosmic-ray neutron sensing for hydrological applications. Frontiers in Water, 2, p.9.
    
    Rossini, P. and Patrignani, A., 2021. Predicting rootzone soil moisture from surface observations in cropland using an exponential filter. 
    Soil Science Society of America Journal.
    """

    # Parameters
    t_delta = 1
    sm_min = df['vwc'].min()
    sm_max = df['vwc'].max()
    ms = (df['vwc']-sm_min)/(sm_max-sm_min)
    
    # Pre-allocate soil water index array
    SWI = np.ones_like(ms)*np.nan
    K = np.ones_like(ms)*np.nan
    
    # Initial conditions
    SWI[0] = ms[0]
    K[0] = 1

    # Values from 2 to N
    for n in range(1,len(SWI)):
        if ~np.isnan(ms[n]) & ~np.isnan(ms[n-1]):
            K[n] = K[n-1] / (K[n-1] + np.exp(-t_delta/T))
            SWI[n] = SWI[n-1] + K[n]*(ms[n] - SWI[n-1])
        else:
            continue

    # Surface storage
    df['storage_surface'] = df['vwc']*Zsurface
    
    # Rootzone storage
    df['storage_rootzone'] = (SWI*(sm_max-sm_min) + sm_min)*Zrootzone
    
    return df
                   
                   

In [35]:
# Define file name, site-specific variables, and load data
file_name = 'hydroinnova_stationary_bf/hydroinnova.csv'
skip_rows = 21
variables = {'timestamp':[2],
             'temperature':[7],
             'humidity':[8],
             'pressure':[3],
             'counts_raw':[10],
             'counts_time':[11]}
number_detectors = 1
count_time = 3600 # seconds
par = {'interval': 60,
       'Pref':980, 
       'L':130, 
       'Aref':0,
       'N0':3486, 
       'Wlat':0.03,
       'Wsoc':0.01, 
       'bulk_density':1.33,
       'utc_offset':-6,
       'nmdb_station':'IRKT'}

# Correct neutron counts
data = load_data(file_name,skip_rows,variables) 
data = fill_counts(data)
data = fill_missing_atm(data)
data = compute_total_counts(data)
data = get_incoming_neutron_flux(data,par)
data = atm_correction(data,par)
data = smooth_counts(data)
data = counts_to_vwc(data)
data = sensing_depth(data,par)
data = compute_storage(data,T=5)

Data retrieved via NMDB are the property of the individual data providers. These data are free for non commercial
use to within the restriction imposed by the providers. If you use such data for your research or applications, please acknowledge
the origin by a sentence like 'We acknowledge the NMDB database (www.nmdb.eu) founded under the European Union's FP7 programme 
(contract no. 213007), and the PIs of individual neutron monitors at: IGY Jungfraujoch 
(Physikalisches Institut, University of Bern, Switzerland)


In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['counts_total_raw'], color='green', legend_label='Nraw')
f.line(data['timestamp'], data['counts_total_corrected'], color='black', legend_label='Ncorrected')
f.line(data['timestamp'], data['counts_total_filtered'], color='tomato', legend_label='Nfiltered')
f.yaxis.axis_label = 'Neutron counts (cph)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['temperature'])
f.line(data['timestamp'], data['humidity'])
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['pressure'])
f.yaxis.axis_label = 'Atmospheric pressure (mbar)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['vwc'])
f.yaxis.axis_label = 'Volumetric water content (cm^3/cm^3)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['sensing_depth_D86_1m'], color='tomato', legend_label='1 m')
f.line(data['timestamp'], data['sensing_depth_D86_150m'], color='blue', legend_label='150 m')
f.yaxis.axis_label = 'Sensing depth (cm)'
f.legend.location = 'bottom_left'
show(f)

In [36]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['storage_surface'])
f.yaxis.axis_label = 'Storage (cmm)'
show(f)

In [37]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['storage_rootzone'])
f.yaxis.axis_label = 'Storage (cmm)'
show(f)